# Aylık Türkiye Hava Değerli Okuma ve Sabit Web Sayfasına Çevirme

## Barış Sanlı, barissanli2@gmail.com

Aşağıdaki kod, bir sürü paralı BI araçlarına para yatıranlar yerine, açık kodlu, internetten doğrudan CSV ile indiren ve etkileşimli grafiklere çeviren yazılımdır

Veri kaynağı: Uluslararası Enerji Ajansı'nın Weather for Energy Data veritabanıdır. Adres değişirse kok değişkenden değiştirebilirsiniz

Kodun aktif halini barissanli.com/enerji 'de Aylık İklim Göstergelerinde bulabilirsiniz

In [1]:
# web adres
# http://weatherforenergydata.iea.org/National%20data/Monthly%20values/CSV/

In [ ]:
import os
import requests
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas as pd

In [ ]:
import plotly.graph_objs as go
import plotly.io as pio

In [ ]:
kok="http://weatherforenergydata.iea.org/National%20data/Monthly%20values/CSV/"
cdd21='IEA_CMCC_CDD21monthlyworldbypopallmonths.csv'
hdd18='IEA_CMCC_HDD18monthlyworldbypopallmonths.csv'
cloud='IEA_CMCC_Cloudmonthlyworldbysurfallmonths.csv'
daylight='IEA_CMCC_Daylightmonthlyworldbypopallmonths.csv'
evaporation='IEA_CMCC_Evaporationmonthlyworldbysurfallmonths.csv'
precipitation='IEA_CMCC_Precipitationmonthlyworldbysurfallmonths.csv'
snowfall='IEA_CMCC_Snowfallmonthlyworldbysurfallmonths.csv'
wind100m='IEA_CMCC_Wind100intmonthlyworldbysurfallmonths.csv'
sicaklik='IEA_CMCC_Temperaturemonthlyworldbypopallmonths.csv'
wetbulb="IEA_CMCC_Temperaturewbmonthlyworldbypopallmonths.csv"

In [ ]:
filez=["cdd21","hdd18","cloud","daylight","evaporation","precipitation","snowfall","wind100m","sicaklik","wetbulb"]

In [ ]:
# Sürekli Web'den indirip durma, eğer dosyalar yoksa ve yenile=True ise yenile
yenile=False

In [ ]:
def geturlandgraph(csvfile,baslik="",yl="",filen=""):
 file_path=eval(i)
 if not os.path.exists(file_path) or yenile:
    # Download the file
    response = requests.get(kok+file_path)
    with open(file_path, 'wb') as file:
        file.write(response.content)
    print(f"Downloaded {file_path}")
    df = pd.read_csv(file_path, na_values=[''],skiprows=9, encoding='unicode_escape')  
 else:
    df = pd.read_csv(file_path, na_values=[''],skiprows=9, encoding='unicode_escape')  
 
 df = pd.read_csv(kok+csvfile, na_values=[''],skiprows=9, encoding='unicode_escape')  
 datavar=df.columns[-1]
 df=df[df.ISO3=="TUR"]
 df["yil"]=df.Date.str[:4].astype(int)
 df["ay"]=df.Date.str[5:7].astype(int) ;
 df["gun"]=df.Date.str[8:10].astype(int) ;
 dfx=df.pivot_table(index="ay", columns="yil", values=datavar)
 graphdf(dfx,title=baslik,yl=yl,filen=filen)

In [ ]:
def graphdf(dfx,title="",yl="",filen=""):
    years=dfx.columns
    lastyear=years[-1]
    otheryears=years[:-1]
    df_oy=dfx[otheryears]
    df_oy["minx"]=df_oy.min(axis=1)
    df_oy["maxx"]=df_oy.max(axis=1)
    
#    print(df_oy.head(6))
    months = list(range(1, 13))

    # Create figure
    fig = go.Figure()

    #fig.update_traces(hovertemplate='<b>%{y}</b><br>%{fullData.name}')

    # Add lines for each year
    for year in otheryears:
     fig.add_trace(go.Scatter(
        x=months, y=df_oy[year],
        mode='lines', name=year,
        line=dict(color='lightgray'),
        #hoverinfo='y, fullData.name'
        hovertemplate='y: %{y}<br>Year: %{fullData.name}<extra></extra>'

     ))
     #fig.update_traces(hovertemplate='<b>%{y}</b><br>%{fullData.name}')

#fig = px.line(df_melted, x='Index', y='Temperature', color='Year', title='Annual Temperature Data')

    min_values=df_oy.minx
    max_values=df_oy.maxx
  
    fig.add_trace(go.Scatter(
     x=months, y=min_values,
     mode='lines', line=dict(width=0), name="En Düşük",
     showlegend=False
    ))

    fig.add_trace(go.Scatter(
     x=months, y=max_values,
     mode='lines', line=dict(width=0), name="En Yüksek",
     fill='tonexty', fillcolor='rgba(200, 200, 200, 0.5)',
     showlegend=False
    ))

    fig.add_trace(go.Scatter(
     x=months, y=dfx[lastyear],
     mode='lines',  line=dict(width=2, color="#990000"),
     showlegend=False, name="2024"
    ))

    for year in otheryears:
     fig.add_trace(go.Scatter(
        x=months, y=df_oy[year],
        mode='lines', name=year,
        line=dict(width=2,color='#990000'),
        #hoverinfo='x',
        visible='legendonly'
    ))

    fig.update_layout(
     title={'text':title,'xanchor': 'center','x': 0.5},

     xaxis_title='Aylar',
     yaxis_title=yl,
    #hovermode='x'
    )
    fig.show()

    if filen!="":   #eğer dosya ismi varsa,  dosyaya yaz
     fig.write_html(filen)
    if filen=="":   #eğer dosya ismi yoksa, daha önce açtığımız dosyaya yaz
     f.write(pio.to_html(fig, full_html=False))


In [ ]:
filez

In [ ]:
txt=dict({'cdd21': 'Soğutma Derece-Gün Sayısı (21 C için)',
          'hdd18': 'Isıtma Derece-Gün Sayısı (18 C için)',
          'cloud': 'Bulutlanma Oranı',
          'daylight': 'Günışığı Süresi',
          'evaporation': 'Buharlaşma',
          'precipitation': 'Yağış',
          'snowfall':'Karyağış Miktarı',
          'wind100m':'100 metrede Rüzgar',
          'sicaklik': 'Sıcaklık',
          'wetbulb':'Islak Sıcaklık (Wetbulb)'
          }
        )

In [ ]:
# tüm grafikleri Turkiye-iklim-gostergeleri.html'e yaz, aynı dizinde

with open('Turkiye-iklim-gostergeleri.html', 'w',encoding='utf-8') as f:
 f.write('<!DOCTYPE html><html lang=\"tr\"><head><meta charset=\"UTF-8\"><meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0\"><meta http-equiv=\"Content-Type\" content=\"text/html; charset=ISO-8859-9\"><meta http-equiv=\"Content-Language\" content=\"tr\">')
 f.write('</head>')
 for i in filez:
    #print(eval(i))
    #geturlandgraph(eval(i),txt[i],txt[i],i+".html")
    geturlandgraph(eval(i),txt[i],txt[i],"")

 f.write('</body></html>')

In [ ]:
#son kontrol
for i in filez:
 print(i+"         "+txt[i])